In [4]:
# imports
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np

# settings view dataframes
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)


# find dataset
data_melb = [
    Path('data') / 'melb_data_fe.csv',
    Path('..') / 'data' / 'melb_data_fe.csv',
    Path('../..') / 'data' / 'melb_data_fe.csv',
]
for p in data_melb:
    if p.is_file():
        DATA_PATH = p
        break
else:
    raise FileNotFoundError('Не найден melb_data_fe.csv')

print('Использую файл:', DATA_PATH.resolve())

# load dataset
melb_data_fe = pd.read_csv(DATA_PATH)

# fast sanity-check
print('Форма набора данных (кол-во строк и столбцов):', melb_data_fe.shape)
print('Первые 5 столбцов:', list(melb_data_fe.columns[:23]))
melb_data_fe.head()

Использую файл: /Users/kirilltishchenko/lessonsPandas/data/melb_data_fe.csv
Форма набора данных (кол-во строк и столбцов): (13580, 26)
Первые 5 столбцов: ['Suburb', 'Rooms', 'Type', 'Price', 'Method', 'SellerG', 'Date', 'Distance', 'Postcode', 'Bedroom', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'CouncilArea', 'Lattitude', 'Longtitude', 'Regionname', 'Propertycount', 'MeanRoomsSquare', 'AreaRatio', 'MonthSale', 'AgeBuilding']


,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,Bathroom,Car,Landsize,BuildingArea,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend
0,Abbotsford,2,house,1480000.0,S,Biggin,2016-03-12,2.5,3067,2,1,1,202.0,126.0,Yarra,-37.7996,144.9984,Northern Metropolitan,4019,25.20,-0.231707,3,46,5,St,1
1,Abbotsford,2,house,1035000.0,S,Biggin,2016-04-02,2.5,3067,2,1,0,156.0,79.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019,15.80,-0.327660,4,116,5,St,1
2,Abbotsford,3,house,1465000.0,SP,Biggin,2017-04-03,2.5,3067,3,2,0,134.0,150.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019,18.75,0.056338,4,117,0,St,0
3,Abbotsford,3,house,850000.0,PI,Biggin,2017-04-03,2.5,3067,3,2,1,94.0,126.0,Yarra,-37.7969,144.9969,Northern Metropolitan,4019,15.75,0.145455,4,47,0,other,0
4,Abbotsford,4,house,1600000.0,VB,Nelson,2016-04-06,2.5,3067,3,1,2,120.0,142.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019,17.75,0.083969,4,2,2,St,0


In [8]:
# гарантируем корректный парсинг дат (в мелбурнском датасете обычно день/месяц)
melb_data_fe['Date'] = pd.to_datetime(melb_data_fe['Date'], errors='coerce', dayfirst=True)

# создаем колонку с кварталом продажи: 1..4
# Важно: после to_datetime с errors='coerce' возможны NaT; используем nullable целочисленный тип 'Int8'
melb_data_fe['quarter'] = melb_data_fe['Date'].dt.quarter.astype('Int8')

# выводим количество продаж по кварталам
second_popular_quaerter = melb_data_fe['quarter'].value_counts(dropna=False)
second_popular_quaerter

quarter
<NA>    8442
3       1989
4       1344
2       1320
1        485
Name: count, dtype: Int64

In [9]:
# Преобразуйте все столбцы, в которых меньше 150 уникальных значений, 
# в тип данных category, исключив из преобразования столбцы Date, Rooms, Bedroom, Bathroom, Car.
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_data_fe.columns: # цикл по именам столбцов
    if melb_data_fe[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_data_fe[col] = melb_data_fe[col].astype('category') # преобразуем тип столбца
#display(melb_data_fe.info())
# выводим список столбцов с типом category
melb_data_fe.select_dtypes(['category']).columns.tolist()

['Suburb',
 'Type',
 'Method',
 'SellerG',
 'CouncilArea',
 'Regionname',
 'MonthSale',
 'WeekdaySale',
 'StreetType',
 'Weekend',
 'quarter']

In [10]:
# Отсортируем таблицу по возрастанию цены объектов недвижимости (Price):

melb_data_fe.sort_values(by='Price').head(10)

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,Bathroom,Car,Landsize,BuildingArea,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend,quarter
2652,Footscray,1,unit,85000.0,PI,Burnham,2016-09-03,6.4,3011,1,1,0,0.0,126.0,Maribyrnong,-37.79110,144.89000,Western Metropolitan,7570,42.000000,1.000000,3,9,2,St,0,3
1805,other,4,house,131000.0,PI,other,NaT,8.9,3162,4,1,2,499.0,155.0,Glen Eira,-37.88640,145.02420,Southern Metropolitan,2379,17.222222,-0.525994,2,97,5,St,1,NaN
7303,Albion,1,unit,145000.0,PI,Biggin,NaT,13.9,3020,2,1,1,36.0,126.0,Brimbank,-37.78330,144.82660,Western Metropolitan,2185,31.500000,0.555556,5,46,5,St,1,NaN
1927,Coburg,4,house,145000.0,PI,Jellis,2016-06-04,7.8,3058,3,1,1,536.0,164.0,Moreland,-37.75550,144.96580,Northern Metropolitan,11204,20.500000,-0.531429,4,106,2,Rd,0,2
7940,Hawthorn,1,unit,160000.0,VB,HAR,2017-04-08,4.6,3122,1,1,0,322.0,126.0,Boroondara,-37.81980,145.03730,Southern Metropolitan,11308,42.000000,-0.437500,8,8,4,St,0,2
12666,Brunswick,1,unit,170000.0,VB,Nelson,NaT,5.2,3056,1,1,0,1250.0,126.0,NaN,-37.77685,144.95188,Northern Metropolitan,11918,42.000000,-0.816860,9,47,5,St,1,NaN
8811,Footscray,1,unit,170000.0,PI,Burnham,2017-07-01,5.1,3011,1,1,0,30.0,26.0,Maribyrnong,-37.80141,144.89587,Western Metropolitan,7570,8.666667,-0.071429,1,4,5,St,1,3
8504,West Footscray,1,unit,185000.0,PI,Jas,NaT,8.2,3012,1,1,1,0.0,126.0,Maribyrnong,-37.79800,144.86720,Western Metropolitan,5058,42.000000,1.000000,4,47,5,St,1,NaN
7293,Albion,1,unit,185000.0,S,hockingstuart,NaT,13.9,3020,1,1,1,2347.0,43.0,Brimbank,-37.78520,144.82720,Western Metropolitan,2185,14.333333,-0.964017,8,41,0,Rd,0,NaN
7305,Albion,2,unit,190000.0,SP,Burnham,NaT,13.9,3020,2,1,1,0.0,126.0,Brimbank,-37.78390,144.82390,Western Metropolitan,2185,25.200000,1.000000,7,46,5,St,1,NaN


In [37]:
# А теперь отсортируем таблицу по убыванию 
# (от самой последней до самой первой) даты продажи объекта (Date). 
# Для этого выставим параметр ascending на False
melb_data_fe.sort_values(by='Date', ascending=False)

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,Bathroom,Car,Landsize,BuildingArea,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend,quarter
11144,Northcote,4,house,1955000.0,SP,McGrath,2017-12-08,5.3,3070,4,2,1,5.0,126.0,Darebin,-37.76280,144.99375,Northern Metropolitan,11364,12.600000,0.923664,12,47,4,St,0,4
11217,Surrey Hills,3,house,1775000.0,PI,Jellis,2017-12-08,10.2,3127,3,2,2,780.0,126.0,Boroondara,-37.81403,145.08968,Southern Metropolitan,5457,15.750000,-0.721854,12,47,4,Rd,0,4
11206,St Kilda,4,house,1600000.0,VB,Gary,2017-12-08,5.0,3182,4,2,3,613.0,160.0,Port Phillip,-37.86667,144.98324,Southern Metropolitan,13240,16.000000,-0.586028,12,107,4,St,0,4
11207,Strathmore,7,house,2000000.0,SP,Nelson,2017-12-08,8.2,3041,7,3,4,608.0,355.0,Moonee Valley,-37.73681,144.90587,Western Metropolitan,3284,20.882353,-0.262721,12,18,4,Rd,0,4
11208,Strathmore,4,house,1610000.0,S,other,2017-12-08,8.2,3041,4,3,2,861.0,274.0,Moonee Valley,-37.73154,144.92243,Western Metropolitan,3284,24.909091,-0.517181,12,62,4,St,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,Camberwell,4,house,2650000.0,S,Jellis,2016-03-09,7.8,3124,4,2,1,652.0,189.0,Boroondara,-37.82860,145.06860,Southern Metropolitan,8920,18.900000,-0.550535,3,126,2,Rd,0,1
4510,Oak Park,3,house,892000.0,S,Nelson,2016-03-09,11.5,3046,3,1,3,757.0,126.0,Moreland,-37.71800,144.91740,Northern Metropolitan,2651,18.000000,-0.714609,3,46,2,St,0,1
4388,Northcote,3,house,1200000.0,S,Nelson,2016-03-09,5.5,3070,3,1,1,215.0,125.0,Darebin,-37.77590,144.99090,Northern Metropolitan,11364,17.857143,-0.264706,3,96,2,St,0,1
6184,Surrey Hills,3,house,1205000.0,S,Fletchers,2016-01-28,11.2,3127,3,1,2,490.0,126.0,Whitehorse,-37.83610,145.10060,Southern Metropolitan,5457,18.000000,-0.590909,1,46,3,Rd,0,1


## Сортировка по значениям нескольких столбцов

Для сортировки по значениям нескольких столбцов необходимо передать названия этих столбцов в параметр by в виде списка. При этом важно обращать внимание на порядок следования столбцов.

In [11]:
# выделим каждую десятую строку из столбцов Distance и Price 
# результирующей таблицы
# Групповая сортировка по значениям нескольких столбцов
melb_data_fe.sort_values(by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']]

,Distance,Price
11428,0.0,387000.0
10512,0.7,600000.0
5727,1.2,485000.0
8671,1.2,595000.0
5736,1.2,740000.0
...,...,...
12011,38.0,680000.0
10673,38.0,810000.0
13429,38.0,1155000.0
11102,41.0,650000.0


Найдём информацию о таунхаусах (Type), проданных компанией (SellerG) McGrath, у которых коэффициент соотношения площадей здания и участка (AreaRatio) меньше -0.8. Результат отсортируем по дате продажи (Date) в порядке возрастания, а после проведём сортировку по убыванию коэффициента соотношения площадей. Также обновим старые индексы на новые, установив параметр ignore_index на True. Для наглядности результата выберем из таблицы только столбцы Data и AreaRatio:

In [12]:
mask1 = melb_data_fe['Type'] == 'townhouse'
mask2 = melb_data_fe['AreaRatio'] < -0.8
mask3 = melb_data_fe['SellerG'] == 'McGrath'
melb_data_fe[mask1 & mask2 & mask3].sort_values(
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]

,Date,AreaRatio
0,2016-11-12,-0.945946
1,2017-04-08,-0.947368
2,2017-04-08,-0.970874
3,NaT,-0.953608
4,NaT,-0.971831
5,NaT,-0.974922


In [13]:
# Создайте новый DataFrame, в который войдут только те строки из melb_df,
# в которых значение в столбце Rooms больше 5.
# Отсортируйте этот DataFrame по возрастанию значений в столбце Rooms.
# Результат выведите на экран, ограничившись первыми пятью строками

filtered = melb_data_fe[melb_data_fe['Rooms'] > 5].copy()

filtered.sort_values(inplace=True, by=['Rooms'])

filtered.head()

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,Bathroom,Car,Landsize,BuildingArea,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend,quarter
47,Airport West,6,house,725000.0,SP,Barry,2016-05-07,13.5,3042,6,3,2,976.0,186.0,Moonee Valley,-37.72390,144.87350,Western Metropolitan,3464,12.4000,-0.679862,7,36,1,Rd,0,2
10043,other,6,house,1050000.0,S,other,NaT,35.4,3198,6,4,2,662.0,373.0,Frankston,-38.10780,145.13050,South-Eastern Metropolitan,8077,23.3125,-0.279227,6,47,5,St,1,NaN
9803,Box Hill,6,house,2150500.0,PI,McGrath,NaT,10.9,3128,6,3,3,786.0,126.0,Whitehorse,-37.82616,145.12748,Eastern Metropolitan,4605,8.4000,-0.723684,6,47,5,St,1,NaN
9591,other,6,house,935000.0,S,other,NaT,18.0,3037,6,4,2,694.0,126.0,Melton,-37.68178,144.73779,Western Metropolitan,5556,7.8750,-0.692683,6,47,5,Dr,1,NaN
9582,Hawthorn East,6,house,2200000.0,VB,Jellis,NaT,6.2,3123,6,3,3,622.0,126.0,Boroondara,-37.83510,145.05542,Southern Metropolitan,6482,8.4000,-0.663102,6,47,5,Rd,1,NaN


In [14]:
# Какое значение площади здания (BuildingArea) у объекта недвижимости,
# который находится на 1558 месте в отсортированной по убыванию площади
# здания (BuildingArea) таблице melb_data_fe? Ответ округлите до целого числа.
sorted_df = melb_data_fe.sort_values(by='AreaRatio', ignore_index=False)

val = sorted_df.loc[1558, 'BuildingArea']        # значение площади здания
val

np.float64(126.0)

In [15]:
# Таунхаусы с Rooms > 2
df = melb_data_fe.loc[(melb_data_fe['Type'] == 'townhouse') & (melb_data_fe['Rooms'] > 2)].copy()

# Сортировка: Rooms по возрастанию, MeanRoomsSquare по убыванию, с новыми индексами
df_sorted = df.sort_values(by=['Rooms', 'MeanRoomsSquare'],
                           ascending=[True, False],
                           ignore_index=True)

# Цена в строке 18
price_18 = int(round(df_sorted.loc[18, 'Price']))
price_18

1300000

## Метод группировки

по коду: группируем.вычсляем.сортируем

Коротко: groupby в pandas работает по схеме Split–Apply–Combine.

Split: разбивает строки на группы по ключу(ям) — значениям столбцов.

Apply: применяет функцию к каждой группе (sum, mean, min, max, count, agg и т. д.).

Combine: склеивает результаты в Series/DataFrame.

Похоже на SQL GROUP BY: равные значения ключей попадают в одну группу. Различия в пробелах/регистре — это разные группы. NaN в ключе по умолчанию исключаются (dropna=True), можно включить: groupby(..., dropna=False).

In [17]:
# Группировка. Средние значения по типам недвижимости
melb_data_fe.groupby(by='Type').mean(numeric_only=True)

/var/folders/3p/fk_fhxnj15d5nfky8xh95zy00000gn/T/ipykernel_9577/1568625937.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  melb_data_fe.groupby(by='Type').mean(numeric_only=True)


,Rooms,Price,Distance,Postcode,Bedroom,Bathroom,Car,Landsize,BuildingArea,Lattitude,Longtitude,Propertycount,MeanRoomsSquare,AreaRatio,AgeBuilding
Type,,,,,,,,,,,,,,,
house,3.260874,1.242665e+06,10.979479,3104.080643,3.229336,1.613822,1.772674,617.181924,152.162553,-37.803795,144.994700,7259.025505,18.996731,-0.490031,55.669700
townhouse,2.837522,9.337351e+05,9.851346,3100.777379,2.814183,1.809695,1.555655,279.606822,134.649710,-37.815782,144.996489,7094.459605,18.569847,-0.094916,26.690305
unit,1.963871,6.051275e+05,7.607391,3110.797481,1.966523,1.183295,1.128936,477.314219,102.235863,-37.823710,144.996363,8199.280080,21.068242,0.319883,39.703016


In [20]:
# Группировка. Средние значения по типам недвижимости
melb_data_fe.groupby('Type')['Price'].mean()


/var/folders/3p/fk_fhxnj15d5nfky8xh95zy00000gn/T/ipykernel_9577/1181691971.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  melb_data_fe.groupby('Type')['Price'].mean()


Type
house        1.242665e+06
townhouse    9.337351e+05
unit         6.051275e+05
Name: Price, dtype: float64

In [22]:
# Минимальное расстояние до центра города по регионам, отсортированное по убыванию
melb_data_fe.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

/var/folders/3p/fk_fhxnj15d5nfky8xh95zy00000gn/T/ipykernel_9577/640854051.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  melb_data_fe.groupby('Regionname')['Distance'].min().sort_values(ascending=False)


Regionname
Western Victoria              29.8
Eastern Victoria              25.2
Northern Victoria             21.8
South-Eastern Metropolitan    14.7
Eastern Metropolitan           7.8
Western Metropolitan           4.3
Southern Metropolitan          0.7
Northern Metropolitan          0.0
Name: Distance, dtype: float64

In [23]:
# Минимальное расстояние до центра города по регионам, отсортированное по убыванию
melb_data_fe.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

/var/folders/3p/fk_fhxnj15d5nfky8xh95zy00000gn/T/ipykernel_9577/640854051.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  melb_data_fe.groupby('Regionname')['Distance'].min().sort_values(ascending=False)


Regionname
Western Victoria              29.8
Eastern Victoria              25.2
Northern Victoria             21.8
South-Eastern Metropolitan    14.7
Eastern Metropolitan           7.8
Western Metropolitan           4.3
Southern Metropolitan          0.7
Northern Metropolitan          0.0
Name: Distance, dtype: float64

Группировка данных по одному критерию с несколькими агрегациями

Чтобы рассчитать несколько агрегирующих методов, можно воспользоваться методом agg(), который принимает список строк с названиями агрегаций.

Давайте построим таблицу для анализа продаж по месяцам. Для этого найдём количество продаж, а также среднее и максимальное значения цен объектов недвижимости (Price), сгруппированных по номеру месяца продажи (MonthSale). Результат отсортируем по количеству продаж в порядке убывания:

In [25]:
melb_data_fe.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)

/var/folders/3p/fk_fhxnj15d5nfky8xh95zy00000gn/T/ipykernel_9577/228216093.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  melb_data_fe.groupby('MonthSale')['Price'].agg(


,count,mean,max
MonthSale,,,
8,1850,1.056371e+06,6500000.0
7,1835,9.314698e+05,9000000.0
5,1644,1.097807e+06,8000000.0
6,1469,1.068981e+06,7650000.0
3,1408,1.146762e+06,5600000.0
4,1246,1.050479e+06,5500000.0
9,1188,1.126349e+06,6400000.0
10,854,1.135970e+06,6250000.0
11,750,1.142503e+06,5050000.0


In [ ]:
# Если вам нужна полная информация обо всех основных статистических 
# характеристиках внутри каждой группы, вы можете воспользоваться 
# методом agg(), передав в качестве его параметра строку 'describe':

melb_data_fe.groupby('MonthSale')['Price'].agg('describe')

/var/folders/3p/fk_fhxnj15d5nfky8xh95zy00000gn/T/ipykernel_9577/3772102095.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  melb_data_fe.groupby('MonthSale')['Price'].agg('describe')


,count,mean,std,min,25%,50%,75%,max
MonthSale,,,,,,,,
1,278.0,9.397921e+05,577668.924214,170000.0,570500.0,795000.0,1111250.0,5200000.0
2,333.0,1.169051e+06,671564.357417,131000.0,710000.0,1020000.0,1478000.0,4735000.0
3,1408.0,1.146762e+06,709573.596867,85000.0,680000.0,945000.0,1400000.0,5600000.0
4,1246.0,1.050479e+06,591892.902979,145000.0,655000.0,905500.0,1298750.0,5500000.0
5,1644.0,1.097807e+06,668492.867996,145000.0,650000.0,905000.0,1371250.0,8000000.0
6,1469.0,1.068981e+06,606010.069052,222000.0,660000.0,900000.0,1325000.0,7650000.0
7,1835.0,9.314698e+05,537390.803161,190000.0,586750.0,800000.0,1150000.0,9000000.0
8,1850.0,1.056371e+06,619617.476541,160000.0,635000.0,892000.0,1310000.0,6500000.0
9,1188.0,1.126349e+06,608734.690742,170000.0,725000.0,980000.0,1360000.0,6400000.0


In [29]:
# подсчёт числа уникальных значений
# Более того, метод agg() поддерживает использование и других функций. 
# Передадим дополнительно встроенную функцию set, чтобы получить 
# множество из агентств недвижимости, которые работают в каждом из регионов:

melb_data_fe.groupby('Regionname')['SellerG'].agg(
    		['nunique', set]
)
    

/var/folders/3p/fk_fhxnj15d5nfky8xh95zy00000gn/T/ipykernel_9577/1747175193.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  melb_data_fe.groupby('Regionname')['SellerG'].agg(


,nunique,set
Regionname,,
Eastern Metropolitan,26,"{C21, Nelson, Love, Jellis, Fletchers, McGrath..."
Eastern Victoria,11,"{C21, Eview, Barry, HAR, Ray, hockingstuart, O..."
Northern Metropolitan,40,"{Greg, C21, McDonald, Nick, Nelson, Love, Hodg..."
Northern Victoria,11,"{LITTLE, McDonald, YPA, Buckingham, Barry, Rai..."
South-Eastern Metropolitan,25,"{Chisholm, Greg, C21, Nelson, Hodges, Jellis, ..."
Southern Metropolitan,38,"{Chisholm, Greg, C21, Nick, Nelson, Hodges, Je..."
Western Metropolitan,34,"{Chisholm, Greg, C21, McDonald, Douglas, Moone..."
Western Victoria,6,"{YPA, Raine, Ray, hockingstuart, other, HAR}"


In [43]:
# Сгруппируйте данные по признаку количества комнат и найдите среднюю цену объектов 
# недвижимости в каждой группе. 
# В качестве ответа запишите количество комнат, для которых средняя цена наибольшая.
bedroom_groups = melb_data_fe.groupby('Bedroom')
bedroom_groups['Price'].mean().idxmax()

# .idxmax() — возвращает метку индекса, где значение максимальное → 
# номер комнат с наибольшей средней ценой.


np.int64(7)

In [44]:
# Какой регион имеет наименьшее стандартное отклонение по географической широте (Lattitude)?
melb_data_fe.groupby('Regionname', observed=False)['Lattitude'] \
            .std() \
            .sort_values(ascending=True)

Regionname
Western Victoria              0.011579
Southern Metropolitan         0.043080
Eastern Metropolitan          0.047890
Northern Metropolitan         0.049639
Western Metropolitan          0.051251
South-Eastern Metropolitan    0.073411
Northern Victoria             0.084455
Eastern Victoria              0.147067
Name: Lattitude, dtype: float64

In [49]:
# 1) Дата → datetime
melb_data_fe['Date'] = pd.to_datetime(melb_data_fe['Date'], errors='coerce', dayfirst=True)

# 2) Период (включительно)
start = pd.Timestamp('2017-05-01')
end   = pd.Timestamp('2017-09-01')
mask = melb_data_fe['Date'].between(start, end, inclusive='both')

# 3) Суммы по продавцу и минимум
dfp = melb_data_fe.loc[mask].copy()
dfp['Price'] = pd.to_numeric(dfp['Price'], errors='coerce')

revenue_by_seller = (dfp
    .groupby('SellerG', observed=True)['Price']
    .sum(min_count=1)          # группы с одними NaN → NaN
    .dropna())

min_seller  = revenue_by_seller.idxmin()
min_revenue = revenue_by_seller.loc[min_seller]

print('Компания с наименьшей выручкой:', min_seller)
print('Выручка:', float(min_revenue))
# при необходимости посмотреть несколько минимальных
display(revenue_by_seller.sort_values().head(10))

Компания с наименьшей выручкой: Alexkarbon
Выручка: 1672000.0


SellerG
Alexkarbon    1672000.0
Burnham       1792500.0
Nick          2000000.0
Bells         2455000.0
LITTLE        2567000.0
Collins       3013000.0
Moonee        3152000.0
Buckingham    3645100.0
Thomson       3722000.0
Rendina       4351000.0
Name: Price, dtype: float64

In [51]:
# Также можно построить таблицу, в которой мы будем учитывать тип здания (Type). 
# Для этого в параметрах метода groupby() укажем список из нескольких интересующих нас столбцов.

melb_data_fe.groupby(['Rooms', 'Type'])['Price'].mean()

/var/folders/3p/fk_fhxnj15d5nfky8xh95zy00000gn/T/ipykernel_9577/3728830815.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  melb_data_fe.groupby(['Rooms', 'Type'])['Price'].mean()


Rooms  Type     
1      house        8.668655e+05
       townhouse    5.927045e+05
       unit         3.899289e+05
2      house        1.017238e+06
       townhouse    7.101585e+05
       unit         6.104905e+05
3      house        1.109233e+06
       townhouse    9.847087e+05
       unit         8.505963e+05
4      house        1.462283e+06
       townhouse    1.217092e+06
       unit         1.037476e+06
5      house        1.877327e+06
       townhouse    1.035000e+06
       unit                  NaN
6      house        1.869508e+06
       townhouse             NaN
       unit         5.200000e+05
7      house        1.920700e+06
       townhouse             NaN
       unit                  NaN
8      house        1.510286e+06
       townhouse             NaN
       unit         2.250000e+06
10     house        9.000000e+05
       townhouse             NaN
       unit                  NaN
Name: Price, dtype: float64

Для того, чтобы финальный результат был представлен в виде сводной таблицы (первый группировочный признак по строкам, а второй — по столбцам), а не в виде Series с иерархическими индексами, к результату чаще всего применяют метод unstack(), который позволяет переопределить вложенный индекс в виде столбцов таблицы:

In [ ]:
melb_data_fe.groupby(['Rooms', 'Type'])['Price'].mean().unstack()

# В результате мы получаем сводную таблицу, столбцы в которой представляют типы домов 
# (house, townhouse, unit), строки — число комнат, а на пересечении строк и 
# столбцов находится средняя стоимость объекта с такими показателями.

/var/folders/3p/fk_fhxnj15d5nfky8xh95zy00000gn/T/ipykernel_9577/646791252.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  melb_data_fe.groupby(['Rooms', 'Type'])['Price'].mean().unstack()


Type,house,townhouse,unit
Rooms,,,
1,8.668655e+05,5.927045e+05,3.899289e+05
2,1.017238e+06,7.101585e+05,6.104905e+05
3,1.109233e+06,9.847087e+05,8.505963e+05
4,1.462283e+06,1.217092e+06,1.037476e+06
5,1.877327e+06,1.035000e+06,NaN
6,1.869508e+06,NaN,5.200000e+05
7,1.920700e+06,NaN,NaN
8,1.510286e+06,NaN,2.250000e+06
10,9.000000e+05,NaN,NaN


## Метод pivot_table для построения сводных таблиц

На самом деле метод groupby редко используется при двух параметрах, так как для построения сводных таблиц существует специальный и более простой метод — pivot_table().

values — имя столбца, по которому необходимо получить сводные данные, применяя агрегирующую функцию;

index — имя столбца, значения которого станут строками сводной таблицы;

columns — имя столбца, значения которого станут столбцами сводной таблицы;

aggfunc — имя или список имён агрегирующих функций (по умолчанию — подсчёт 
среднего, 'mean');

fill_value — значение, которым необходимо заполнить пропуски (по умолчанию пропуски не заполняются).

Давайте построим ту же самую таблицу, но уже с использованием метода pivot_table. В качестве параметра values укажем столбец Price, в качестве индексов сводной таблицы возьмём Rooms, а в качестве столбцов — Type. Агрегирующую функцию оставим по умолчанию (среднее). Дополнительно заменим пропуски в таблице на значение 0. Финальный результат для наглядности вывода округлим с помощью метода round() до целых.

In [54]:
melb_data_fe.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type',
    fill_value=0
).round()

/var/folders/3p/fk_fhxnj15d5nfky8xh95zy00000gn/T/ipykernel_9577/4292017364.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  melb_data_fe.pivot_table(


Type,house,townhouse,unit
Rooms,,,
1,866866.0,592705.0,389929.0
2,1017238.0,710158.0,610491.0
3,1109233.0,984709.0,850596.0
4,1462283.0,1217092.0,1037476.0
5,1877327.0,1035000.0,0.0
6,1869508.0,0.0,520000.0
7,1920700.0,0.0,0.0
8,1510286.0,0.0,2250000.0
10,900000.0,0.0,0.0


А теперь давайте проанализируем продажи в каждом из регионов в зависимости от того, будний был день или выходной. Для этого построим сводную таблицу, в которой строками будут являться названия регионов (Regionname), а в столбцах будет располагаться наш «признак-мигалка» выходного дня (Weekend), который равен 1, если день был выходным, и 0 — в противном случае. В качестве значений сводной таблицы возьмём количество продаж.

In [55]:
melb_data_fe.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='count'
)

/var/folders/3p/fk_fhxnj15d5nfky8xh95zy00000gn/T/ipykernel_9577/3197546122.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  melb_data_fe.pivot_table(


Weekend,0,1
Regionname,,
Eastern Metropolitan,447,1024
Eastern Victoria,13,40
Northern Metropolitan,1258,2632
Northern Victoria,11,30
South-Eastern Metropolitan,123,327
Southern Metropolitan,1534,3161
Western Metropolitan,960,1988
Western Victoria,8,24


Разберём ещё один пример: найдём, как зависит средняя и медианная площадь участка (Landsize) от типа объекта (Type) и его региона (Regionname). Чтобы посмотреть несколько статистических параметров, нужно передать в аргумент aggfunc список из агрегирующих функций. Построим такую сводную таблицу, где пропущенные значения заменим на 0:

In [ ]:
melb_data_fe.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)


# Здесь в глаза бросаются объекты типа house в регионах Eastern Victoria и 
# Northern Victoria — в них среднее и медиана отличаются более чем в три раза. 
# Вероятно, это связано с тем, что в этих районах очень большой разброс цен: 
# есть несколько объектов с гигантской площадью, а остальные объекты имеют 
# небольшую площадь. Из-за этого среднее значение искажается, в то время как 
# медиана нечувствительна к такому разбросу и не искажает результат.

/var/folders/3p/fk_fhxnj15d5nfky8xh95zy00000gn/T/ipykernel_9577/2813588845.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  melb_data_fe.pivot_table(
/var/folders/3p/fk_fhxnj15d5nfky8xh95zy00000gn/T/ipykernel_9577/2813588845.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  melb_data_fe.pivot_table(


median                          mean                        
Type                        house townhouse   unit        house   townhouse        unit
Regionname                                                                             
Eastern Metropolitan        674.0     233.5  203.0   717.422847  269.440678  330.444444
Eastern Victoria            843.0       0.0  230.0  3108.960000    0.000000  295.333333
Northern Metropolitan       459.5     134.0    0.0   619.249092  317.325733  495.026538
Northern Victoria           724.0       0.0    0.0  3355.463415    0.000000    0.000000
South-Eastern Metropolitan  630.5     240.0  199.0   664.306701  212.160000  357.864865
Southern Metropolitan       586.0     246.0    0.0   569.643881  278.858824  466.380245
Western Metropolitan        531.0     198.0   62.0   507.883406  244.560669  557.637232
Western Victoria            599.5       0.0    0.0   655.500000    0.000000    0.000000

## Многомерные сводные таблицы

До этого мы рассматривали, как некоторый статистический показатель может зависеть от двух признаков. Однако, как уже упоминалось, сводные таблицы позволяют наблюдать зависимость и от большего числа признаков. Такие сводные таблицы называются многомерными. 

Для того чтобы исследовать зависимость от большего числа признаков, можно передать список признаков в параметр index или параметр columns.

In [58]:
# Давайте построим таблицу, в которой по индексам будут располагаться признаки 
# метода продажи (Method) и типа объекта (Type), по столбцам — наименование 
# региона (Regionname), а на пересечении строк и столбцов будет стоять 
# медианная цена объекта (Price):

melb_data_fe.pivot_table(
    values='Price',
    index=['Method','Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
)

/var/folders/3p/fk_fhxnj15d5nfky8xh95zy00000gn/T/ipykernel_9577/3707049719.py:6: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  melb_data_fe.pivot_table(


Regionname        Eastern Metropolitan  Eastern Victoria  Northern Metropolitan  Northern Victoria  \
Method Type                                                                                          
PI     house                 1244000.0          780000.0               900000.0           500000.0   
       townhouse              760000.0               0.0               632500.0                0.0   
       unit                   650000.0               0.0               410000.0                0.0   
S      house                 1127000.0          675000.0               920000.0           555000.0   
       townhouse              828000.0               0.0               750000.0                0.0   
       unit                   645750.0          492000.0               525500.0                0.0   
SA     house                  932500.0          950000.0               817500.0           540000.0   
       townhouse              807500.0               0.0               425000.0                0.0   
       unit                        0.0               0.0               616000.0                0.0   
SP     house                 1050000.0          672500.0               900000.0           521000.0   
       townhouse              910000.0               0.0               690000.0                0.0   
       unit                   515000.0          400000.0               470000.0                0.0   
VB     house                 1100000.0          712500.0              1050000.0           690000.0   
       townhouse              892500.0               0.0               640000.0                0.0   
       unit                   500000.0               0.0               450000.0                0.0   

Regionname        South-Eastern Metropolitan  Southern Metropolitan  Western Metropolitan  Western Victoria  
Method Type                                                                                                  
PI     house                        865000.0              1725000.0              870000.0          630000.0  
       townhouse                   1190000.0              1055000.0              670000.0               0.0  
       unit                         525000.0               571250.0              360000.0               0.0  
S      house                        883300.0              1611000.0              870000.0          397500.0  
       townhouse                    875000.0              1135000.0              729000.0               0.0  
       unit                         606000.0               655000.0              489000.0               0.0  
SA     house                        880000.0              1390000.0              772500.0               0.0  
       townhouse                         0.0              1141000.0              467500.0               0.0  
       unit                              0.0               580000.0              571000.0               0.0  
SP     house                        770000.0              1521750.0              865000.0          360000.0  
       townhouse                    800000.0              1162500.0              702500.0               0.0  
       unit                         601000.0               550000.0              460000.0               0.0  
VB     house                        850000.0              1800000.0              880000.0               0.0  
       townhouse                         0.0              1250000.0              689500.0               0.0  
       unit                         700000.0               500000.0              420000.0               0.0

In [59]:
# Запишем сводную таблицу, которую мы создавали ранее в переменную pivot:

pivot = melb_data_fe.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

# Выведем её столбцы с помощью атрибута columns:

pivot.columns

/var/folders/3p/fk_fhxnj15d5nfky8xh95zy00000gn/T/ipykernel_9577/962741114.py:3: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = melb_data_fe.pivot_table(
/var/folders/3p/fk_fhxnj15d5nfky8xh95zy00000gn/T/ipykernel_9577/962741114.py:3: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = melb_data_fe.pivot_table(


MultiIndex([('median',     'house'),
            ('median', 'townhouse'),
            ('median',      'unit'),
            (  'mean',     'house'),
            (  'mean', 'townhouse'),
            (  'mean',      'unit')],
           names=[None, 'Type'])

В результате мы получаем объект MultiIndex. Этот объект хранит в себе шесть комбинаций пар столбцов (два статистических параметра и три типа здания), то есть есть шесть возможных вариантов обращения к столбцам таблицы.

Мультииндексы раскрываются подобно вложенным словарям — по очереди, как матрёшка. Чтобы получить доступ к определённому столбцу, вы должны сначала обратиться к столбцу, который находится уровнем выше.

In [60]:
# Так, из таблицы pivot мы можем получить средние значения площадей участков для 
# типа здания unit, просто последовательно обратившись по имени столбцов:

display(pivot['mean']['unit'])

Regionname
Eastern Metropolitan          330.444444
Eastern Victoria              295.333333
Northern Metropolitan         495.026538
Northern Victoria               0.000000
South-Eastern Metropolitan    357.864865
Southern Metropolitan         466.380245
Western Metropolitan          557.637232
Western Victoria                0.000000
Name: unit, dtype: float64

In [61]:
# Аналогично производится и фильтрация данных. Например, если нам нужны регионы, 
# в которых средняя площадь здания для домов типа house меньше их медианной площади, 
# то мы можем найти их следующим образом:

mask = pivot['mean']['house'] < pivot['median']['house']
filtered_pivot = pivot[mask]
display(filtered_pivot)

median                        mean                        
Type                   house townhouse  unit       house   townhouse        unit
Regionname                                                                      
Southern Metropolitan  586.0     246.0   0.0  569.643881  278.858824  466.380245
Western Metropolitan   531.0     198.0  62.0  507.883406  244.560669  557.637232

In [63]:
# Примечание. На самом деле мультииндексные таблицы можно создавать и вручную. 
# Давайте посмотрим на синтаксис данной конструкции:

mser = pd.Series(
    np.random.rand(8),
	index=[['white','white','white','blue','blue','red','red','red'], 
           ['up','down','right','up','down','up','down','left']])
display(mser)

# В данном примере мы создаём объект Series со вложенными индексами. Мы передаём в 
# качестве индексов Series вложенный список, где первый список задаёт внешний уровень 
# вложенности, а второй список — внутренний уровень вложенности. Значения Series — 
# случайные числа от 0 до 1

white  up       0.279676
       down     0.856271
       right    0.535980
blue   up       0.206725
       down     0.754432
red    up       0.552524
       down     0.620566
       left     0.561091
dtype: float64

In [64]:
# Аналогично создаются DataFrame со вложенными признаками (вложенными столбцами) — 
# для этого вложенный список передаётся в параметр columns при инициализации таблицы:

mframe = pd.DataFrame(
    np.random.randn(16).reshape(4,4),
    index=[['white','white','red','red'], ['up','down','up','down']],
    columns=[['pen','pen','paper','paper'],[1,2,1,2]]
)
display(mframe)

pen               paper          
                   1         2         1         2
white up   -0.018488 -0.329693  0.372127 -1.336906
      down -0.074309 -0.645974 -0.427030  0.046318
red   up   -0.226160  1.804477  0.730359  0.486512
      down  0.178555  0.041496 -2.002999  0.227092

In [69]:
# Составьте сводную таблицу, которая показывает зависимость медианной площади (BuildingArea) здания от типа объекта недвижимости (Type) и количества жилых комнат в доме (Rooms). Для какой комбинации признаков площадь здания наибольшая?
#В качестве ответа запишите эту комбинацию (тип здания, число комнат) через запятую, без пробелов.

pivot_median = melb_data_fe.pivot_table(
    values='BuildingArea',
    index='Type',
    columns='Rooms',
    aggfunc='median',
    fill_value=0
)

max_value = pivot_median.max().max()
max_combination = pivot_median.stack().idxmax()

print(max_value, max_combination)

216.5 ('house', np.int64(7))


/var/folders/3p/fk_fhxnj15d5nfky8xh95zy00000gn/T/ipykernel_9577/2514399033.py:4: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_median = melb_data_fe.pivot_table(


In [73]:
# Составьте сводную таблицу, которая показывает зависимость медианной цены объекта 
# недвижимости (Price) от риелторского агентства (SellerG) и типа здания (Type).
# Во вновь созданной таблице найдите агентство, у которого медианная цена для зданий 
# типа unit максимальна. В качестве ответа запишите название этого агентства.
pivot_price = melb_data_fe.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    aggfunc='median',
    fill_value=0
)

# Во вновь созданной таблице найдите агентство, у которого медианная цена для зданий 
# типа unit максимальна. В качестве ответа запишите название этого агентства.
max_agency = pivot_price['unit'].idxmax()

max_agency

/var/folders/3p/fk_fhxnj15d5nfky8xh95zy00000gn/T/ipykernel_9577/3700211108.py:5: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_price = melb_data_fe.pivot_table(


'Nick'